In [58]:
import pandas as pd
import numpy as np
import re
from re import sub
import multiprocessing
from unidecode import unidecode

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors


from time import time 
from collections import defaultdict

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [2]:
df = pd.read_csv('MOH_Comments_emotions1.csv')
df.head()

,Unnamed: 0,object_id,id,text,created_at,like_count,comment_count,Emoji 1,Emoji 2,Emoji 3,Emoji 4,Emoji 5
0,0,154909330630_214883637333734,214883637333734_214899840665447,And the bad news: you won't know if you have i...,2021-09-04T08:15:55+0000,0.0,1.0,😖,😣,😓,😞,😕
1,1,154909330630_214883637333734,214883637333734_214897943998970,"If people are expressing concerns, then it is ...",2021-09-04T08:11:13+0000,7.0,2.0,😌,👌,😉,👀,💯
2,2,154909330630_214883637333734,214883637333734_214892550666176,How about our kids under 12? They got NO prote...,2021-09-04T08:00:19+0000,16.0,7.0,😬,😳,😑,😐,😕
3,3,154909330630_214883637333734,214883637333734_214902850665146,My past reading n understanding is that now ou...,2021-09-04T08:22:41+0000,1.0,2.0,😷,😐,😕,😠,😡
4,4,154909330630_214883637333734,214883637333734_214941327327965,Protect below 12 year olds: Don't sign them up...,2021-09-04T09:57:54+0000,1.0,1.0,😡,😠,👊,🔫,🙅


In [3]:
df.drop("Unnamed: 0",axis=1)

,object_id,id,text,created_at,like_count,comment_count,Emoji 1,Emoji 2,Emoji 3,Emoji 4,Emoji 5
0,154909330630_214883637333734,214883637333734_214899840665447,And the bad news: you won't know if you have i...,2021-09-04T08:15:55+0000,0.0,1.0,😖,😣,😓,😞,😕
1,154909330630_214883637333734,214883637333734_214897943998970,"If people are expressing concerns, then it is ...",2021-09-04T08:11:13+0000,7.0,2.0,😌,👌,😉,👀,💯
2,154909330630_214883637333734,214883637333734_214892550666176,How about our kids under 12? They got NO prote...,2021-09-04T08:00:19+0000,16.0,7.0,😬,😳,😑,😐,😕
3,154909330630_214883637333734,214883637333734_214902850665146,My past reading n understanding is that now ou...,2021-09-04T08:22:41+0000,1.0,2.0,😷,😐,😕,😠,😡
4,154909330630_214883637333734,214883637333734_214941327327965,Protect below 12 year olds: Don't sign them up...,2021-09-04T09:57:54+0000,1.0,1.0,😡,😠,👊,🔫,🙅
...,...,...,...,...,...,...,...,...,...,...,...
96181,154909330630_10156730596595631,10156730596595631_10156730619080631,1. Skip the sweet sauce for popiah. It is tast...,2019-01-20T11:34:10+0000,0.0,0.0,😷,😜,👍,💓,😋
96182,154909330630_10156730596595631,10156730596595631_10156740495270631,Also educate the true source of health hazards...,2019-01-24T23:36:28+0000,0.0,0.0,✌,💪,✨,👏,👍
96183,154909330630_10156730596595631,10156730596595631_10156730942510631,https://www.diabetes.org.uk/guide-to-diabetes/...,2019-01-20T15:03:16+0000,0.0,0.0,💪,💛,🔫,✌,💓
96184,154909330630_10156688606900631,10156688606900631_10156689073615631,fine how much? or jus warning shiok shiok only...,2019-01-01T13:58:27+0000,0.0,0.0,🙅,😉,😏,😜,🎶


In [4]:
print(df.shape)

(96186, 12)


In [5]:
df['text'] = df['text'].astype('str')

In [6]:
df1 = pd.DataFrame(df[['text', 'created_at']])
#df1 = pd.DataFrame(df['text'])

In [7]:
df1.dropna(subset = ["text"], inplace=True)
df1.head()
#df_cleaned = df1.dropna().drop_duplicates().reset_index(drop=True)

INFO - 03:30:07: NumExpr defaulting to 8 threads.


,text,created_at
0,And the bad news: you won't know if you have i...,2021-09-04T08:15:55+0000
1,"If people are expressing concerns, then it is ...",2021-09-04T08:11:13+0000
2,How about our kids under 12? They got NO prote...,2021-09-04T08:00:19+0000
3,My past reading n understanding is that now ou...,2021-09-04T08:22:41+0000
4,Protect below 12 year olds: Don't sign them up...,2021-09-04T09:57:54+0000


In [97]:
df_cleaned = df1.drop_duplicates('text').reset_index(drop=True)

In [98]:
df_cleaned

,text,created_at
0,And the bad news: you won't know if you have i...,2021-09-04T08:15:55+0000
1,"If people are expressing concerns, then it is ...",2021-09-04T08:11:13+0000
2,How about our kids under 12? They got NO prote...,2021-09-04T08:00:19+0000
3,My past reading n understanding is that now ou...,2021-09-04T08:22:41+0000
4,Protect below 12 year olds: Don't sign them up...,2021-09-04T09:57:54+0000
...,...,...
78958,1. Skip the sweet sauce for popiah. It is tast...,2019-01-20T11:34:10+0000
78959,Also educate the true source of health hazards...,2019-01-24T23:36:28+0000
78960,https://www.diabetes.org.uk/guide-to-diabetes/...,2019-01-20T15:03:16+0000
78961,fine how much? or jus warning shiok shiok only...,2019-01-01T13:58:27+0000


In [99]:
import pycld2
eng_text = []
for i in range (len(df_cleaned)):
    en_comment = df_cleaned['text'][i]
    try:
        _, _, languages = pycld2.detect(en_comment.strip().encode('utf-8'))
    except:
        languages = []
    #print(languages)
    if len(list(languages)) > 0:
        language1 = languages[0]
        _, lang_code1, _, _ = language1
        language2 = languages[1]
        _, lang_code2, _, _ = language2
        language3 = languages[2]
        _, lang_code3, _, _ = language3
        if lang_code2 == 'un':
            if lang_code1 != 'en':
                #clean_df.append(df_cleaned[i])
                df_cleaned.drop(i, inplace=True)
        else:
            df_cleaned.drop(i, inplace=True)
    else:
        lang_code = 'un'
        df_cleaned.drop(i, inplace=True)

In [100]:
df_cleaned

,text,created_at
0,And the bad news: you won't know if you have i...,2021-09-04T08:15:55+0000
1,"If people are expressing concerns, then it is ...",2021-09-04T08:11:13+0000
2,How about our kids under 12? They got NO prote...,2021-09-04T08:00:19+0000
3,My past reading n understanding is that now ou...,2021-09-04T08:22:41+0000
4,Protect below 12 year olds: Don't sign them up...,2021-09-04T09:57:54+0000
...,...,...
78958,1. Skip the sweet sauce for popiah. It is tast...,2019-01-20T11:34:10+0000
78959,Also educate the true source of health hazards...,2019-01-24T23:36:28+0000
78960,https://www.diabetes.org.uk/guide-to-diabetes/...,2019-01-20T15:03:16+0000
78961,fine how much? or jus warning shiok shiok only...,2019-01-01T13:58:27+0000


In [65]:
df_cleaned = df_cleaned.reset_index(drop=True)

In [73]:
df_cleaned

,text,created_at
0,And the bad news: you won't know if you have i...,2021-09-04T08:15:55+0000
1,"If people are expressing concerns, then it is ...",2021-09-04T08:11:13+0000
2,How about our kids under 12? They got NO prote...,2021-09-04T08:00:19+0000
3,My past reading n understanding is that now ou...,2021-09-04T08:22:41+0000
4,Protect below 12 year olds: Don't sign them up...,2021-09-04T09:57:54+0000
...,...,...
57120,1. Skip the sweet sauce for popiah. It is tast...,2019-01-20T11:34:10+0000
57121,Also educate the true source of health hazards...,2019-01-24T23:36:28+0000
57122,https://www.diabetes.org.uk/guide-to-diabetes/...,2019-01-20T15:03:16+0000
57123,fine how much? or jus warning shiok shiok only...,2019-01-01T13:58:27+0000


In [74]:
def text_to_word_list(text, remove_non_eng):
    
    text = text.lower()
# Clean the text
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = re.sub(r"\+", " plus ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\?", " ? ", text)
    #text = sub(r"'", " ", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"\s{2,}", " ", text)

    #x = text.split()
    
    return text

In [75]:
df_cleaned['text'] = df_cleaned['text'].apply(lambda x: text_to_word_list(x, unidecode))

In [76]:
df_cleaned.head()

,text,created_at
0,and the bad news you won't know if you have it...,2021-09-04T08:15:55+0000
1,if people are expressing concerns then it is a...,2021-09-04T08:11:13+0000
2,how about our kids under 12 ? they got no prot...,2021-09-04T08:00:19+0000
3,my past reading n understanding is that now ou...,2021-09-04T08:22:41+0000
4,protect below 12 year olds don't sign them up ...,2021-09-04T09:57:54+0000


In [77]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long, the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [101]:
stopwords = nlp.Defaults.stop_words

In [102]:
print(len(stopwords))
print(stopwords)

326
{'therefore', 'somehow', 'nobody', 'under', 'my', 'along', 'what', 'thereby', 'whither', 'nine', 'anyway', 'six', 'beforehand', 'n’t', 'onto', 'using', 'thru', 'nothing', 'twelve', 'everywhere', 'amongst', 'via', 'your', 'one', 'more', 'there', 'whose', 'the', 'whoever', 'really', 'whether', 'always', 'due', 'noone', 'front', "'re", 'go', 'thereupon', 'doing', 'become', 'get', "'ll", 'with', 'we', 'empty', 'same', 'such', 'again', 'now', 'few', 'please', 'her', 'hers', 'me', 'mine', 'each', 'hundred', 'out', 'as', 'anywhere', 'namely', 'often', 'whereupon', 'move', 'their', 'ours', 'through', 'top', 'only', 'within', 'any', 'ten', 'if', 'either', 'not', 'see', '‘re', 'how', 'those', 'about', 'even', 'us', '‘m', 'alone', "'ve", 'is', '‘ve', 'also', 'on', "'s", 'were', 'both', 'latterly', 'here', 'it', 'anything', 'until', 'these', 'several', 'four', 'everyone', 'three', 'should', 'was', 'fifteen', 'nowhere', 'perhaps', 'so', 'moreover', 'may', 'did', 'fifty', 'someone', 'most', 'its

In [78]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df_cleaned['text'])

In [79]:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.84 mins


In [80]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(49832, 1)

In [82]:
df_clean = df_clean.reset_index(drop=True)

In [83]:
df_clean

,clean
0,bad news will know test people complacent feel...
1,people express concern indication look deep
2,kid get protection mask ensure safety esp scho...
3,past read n understanding infected individual ...
4,protect year old sign preschool
...,...
49827,enforce drink offer vend machine sugar free of...
49828,skip sweet sauce popiah tasty chilli garlic ...
49829,educate true source health hazard alternative
49830,fine jus warning shiok shiok


In [84]:
sent = [row.split() for row in df_clean.clean]
phrases = Phrases(sent, min_count=1, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

INFO - 02:12:33: collecting all words and their counts
INFO - 02:12:33: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 02:12:33: PROGRESS: at sentence #10000, processed 161741 words and 112780 word types
INFO - 02:12:34: PROGRESS: at sentence #20000, processed 331091 words and 203076 word types
INFO - 02:12:34: PROGRESS: at sentence #30000, processed 463553 words and 262542 word types
INFO - 02:12:34: PROGRESS: at sentence #40000, processed 604624 words and 324102 word types
INFO - 02:12:35: collected 404423 word types from a corpus of 785770 words (unigram + bigrams) and 49832 sentences
INFO - 02:12:35: using 404423 counts as vocab in Phrases<0 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 02:12:35: source_vocab length 404423
INFO - 02:12:38: Phraser built with 21667 phrasegrams


['people', 'express_concern', 'indication', 'look', 'deep']

In [12]:
#df_cleaned['text'] = df_cleaned['text'].astype('str')

In [43]:
df_cleaned

,text,created_at
0,"[and, the, bad, news, you, won't, know, if, yo...",2021-09-04T08:15:55+0000
1,"[if, people, are, expressing, concerns, then, ...",2021-09-04T08:11:13+0000
2,"[how, about, our, kids, under, 12, ?, they, go...",2021-09-04T08:00:19+0000
3,"[my, past, reading, n, understanding, is, that...",2021-09-04T08:22:41+0000
4,"[protect, below, 12, year, olds, don't, sign, ...",2021-09-04T09:57:54+0000
...,...,...
57120,"[1, skip, the, sweet, sauce, for, popiah, it, ...",2019-01-20T11:34:10+0000
57121,"[also, educate, the, true, source, of, health,...",2019-01-24T23:36:28+0000
57122,[],2019-01-20T15:03:16+0000
57123,"[fine, how, much, ?, or, jus, warning, shiok, ...",2019-01-01T13:58:27+0000


In [30]:
#df_cleaned1 = df_cleaned.drop(df_cleaned.index[df_cleaned['text'] == '[]'])

In [93]:
df_model = df_clean.copy()
df_model = df_model[df_model.clean.str.len()>1]

In [44]:
df_model = df_cleaned.copy()
df_model = df_model[df_model.text.str.len()>1]

In [94]:
df_model = df_model.reset_index(drop=True)

In [95]:
df_model

,clean
0,bad news will know test people complacent feel...
1,people express concern indication look deep
2,kid get protection mask ensure safety esp scho...
3,past read n understanding infected individual ...
4,protect year old sign preschool
...,...
49827,enforce drink offer vend machine sugar free of...
49828,skip sweet sauce popiah tasty chilli garlic ...
49829,educate true source health hazard alternative
49830,fine jus warning shiok shiok


In [47]:
df_model.head(30)

,text,created_at
0,"[and, the, bad, news, you, won't, know, if, yo...",2021-09-04T08:15:55+0000
1,"[if, people, are, expressing, concerns, then, ...",2021-09-04T08:11:13+0000
2,"[how, about, our, kids, under, 12, ?, they, go...",2021-09-04T08:00:19+0000
3,"[my, past, reading, n, understanding, is, that...",2021-09-04T08:22:41+0000
4,"[protect, below, 12, year, olds, don't, sign, ...",2021-09-04T09:57:54+0000
5,"[brilliant, stat, to, be, sharing, big, messag...",2021-09-04T12:07:53+0000
6,"[i'm, only, concerned, y, are, we, still, not,...",2021-09-04T09:36:00+0000
7,"[it, s, a, good, development, moh, is, express...",2021-09-04T07:53:02+0000
8,"[awesome, !, that's, right, !, i, hope, those,...",2021-09-04T09:01:45+0000
9,"[this, morning, i, happened, to, be, at, shunf...",2021-09-04T08:35:53+0000


In [48]:
sent = [row for row in df_model.text]
phrases = Phrases(sent, min_count=1, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

INFO - 01:01:47: collecting all words and their counts
INFO - 01:01:47: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 01:01:48: PROGRESS: at sentence #10000, processed 326081 words and 150313 word types
INFO - 01:01:48: PROGRESS: at sentence #20000, processed 657978 words and 254392 word types
INFO - 01:01:49: PROGRESS: at sentence #30000, processed 925020 words and 323760 word types
INFO - 01:01:49: PROGRESS: at sentence #40000, processed 1153465 words and 376746 word types
INFO - 01:01:49: PROGRESS: at sentence #50000, processed 1465501 words and 450543 word types
INFO - 01:01:50: collected 505810 word types from a corpus of 1709855 words (unigram + bigrams) and 56290 sentences
INFO - 01:01:50: using 505810 counts as vocab in Phrases<0 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 01:01:50: source_vocab length 505810
INFO - 01:01:54: Phraser built with 25096 phrasegrams


['if',
 'people',
 'are',
 'expressing_concerns',
 'then',
 'it',
 'is',
 'an_indication',
 'to',
 'look_into',
 'it',
 'deeper']

    min count = 3 - remove most unusual words from training embeddings, like words 'ssssuuuuuuuppppppeeeeeerrrr', which actually stands for 'super', and doesn't need additional training
    window = 4 - Word2Vec model will learn to predict given word from up to 4 words to the left, and up to 4 words to the right
    size = 300 - size of hidden layer used to predict surroundings of embedded word, which also stands for dimensions of trained embeddings
    sample = 1e-5 - probability baseline for subsampling most frequent words from surrounding of embedded word
    negative = 20 - number of negative (ones that shouldn't have been predicted while modeling selected pair of words) words that will have their corresponding weights updated while training on specific training example, along with positive word

In [85]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

INFO - 02:13:30: collecting all words and their counts
INFO - 02:13:30: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 02:13:30: PROGRESS: at sentence #10000, processed 137047 words, keeping 17589 word types
INFO - 02:13:31: PROGRESS: at sentence #20000, processed 279927 words, keeping 26710 word types
INFO - 02:13:31: PROGRESS: at sentence #30000, processed 392269 words, keeping 32201 word types
INFO - 02:13:31: PROGRESS: at sentence #40000, processed 513027 words, keeping 36912 word types
INFO - 02:13:32: collected 42160 word types from a corpus of 666251 raw words and 49832 sentences
INFO - 02:13:32: Loading a fresh vocabulary
INFO - 02:13:32: effective_min_count=3 retains 17339 unique words (41% of original 42160, drops 24821)
INFO - 02:13:32: effective_min_count=3 leaves 630137 word corpus (94% of original 666251, drops 36114)
INFO - 02:13:32: deleting the raw counts dictionary of 42160 items
INFO - 02:13:32: sample=1e-05 downsamples 3878 most-common word

Time to build vocab: 0.08 mins


In [86]:
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

INFO - 02:13:51: training model with 7 workers on 17339 vocabulary and 300 features, using sg=0 hs=0 sample=1e-05 negative=20 window=4
INFO - 02:13:52: EPOCH 1 - PROGRESS: at 51.69% examples, 100658 words/s, in_qsize 0, out_qsize 1
INFO - 02:13:53: worker thread finished; awaiting finish of 6 more threads
INFO - 02:13:53: worker thread finished; awaiting finish of 5 more threads
INFO - 02:13:53: worker thread finished; awaiting finish of 4 more threads
INFO - 02:13:53: worker thread finished; awaiting finish of 3 more threads
INFO - 02:13:53: worker thread finished; awaiting finish of 2 more threads
INFO - 02:13:53: worker thread finished; awaiting finish of 1 more threads
INFO - 02:13:53: worker thread finished; awaiting finish of 0 more threads
INFO - 02:13:53: EPOCH - 1 : training on 666251 raw words (190040 effective words) took 1.9s, 102436 effective words/s
INFO - 02:13:54: EPOCH 2 - PROGRESS: at 55.85% examples, 105003 words/s, in_qsize 0, out_qsize 0
INFO - 02:13:55: worker thr

INFO - 02:14:12: EPOCH 12 - PROGRESS: at 53.76% examples, 102238 words/s, in_qsize 0, out_qsize 0
INFO - 02:14:13: worker thread finished; awaiting finish of 6 more threads
INFO - 02:14:13: worker thread finished; awaiting finish of 5 more threads
INFO - 02:14:13: worker thread finished; awaiting finish of 4 more threads
INFO - 02:14:13: worker thread finished; awaiting finish of 3 more threads
INFO - 02:14:13: worker thread finished; awaiting finish of 2 more threads
INFO - 02:14:13: worker thread finished; awaiting finish of 1 more threads
INFO - 02:14:13: worker thread finished; awaiting finish of 0 more threads
INFO - 02:14:13: EPOCH - 12 : training on 666251 raw words (189829 effective words) took 1.9s, 100203 effective words/s
INFO - 02:14:14: EPOCH 13 - PROGRESS: at 51.69% examples, 99537 words/s, in_qsize 1, out_qsize 0
INFO - 02:14:15: worker thread finished; awaiting finish of 6 more threads
INFO - 02:14:15: worker thread finished; awaiting finish of 5 more threads
INFO - 02:

INFO - 02:14:33: worker thread finished; awaiting finish of 6 more threads
INFO - 02:14:33: worker thread finished; awaiting finish of 5 more threads
INFO - 02:14:33: worker thread finished; awaiting finish of 4 more threads
INFO - 02:14:33: worker thread finished; awaiting finish of 3 more threads
INFO - 02:14:33: worker thread finished; awaiting finish of 2 more threads
INFO - 02:14:33: worker thread finished; awaiting finish of 1 more threads
INFO - 02:14:33: worker thread finished; awaiting finish of 0 more threads
INFO - 02:14:33: EPOCH - 23 : training on 666251 raw words (189720 effective words) took 2.0s, 95232 effective words/s
INFO - 02:14:34: EPOCH 24 - PROGRESS: at 61.71% examples, 110213 words/s, in_qsize 0, out_qsize 0
INFO - 02:14:34: worker thread finished; awaiting finish of 6 more threads
INFO - 02:14:34: worker thread finished; awaiting finish of 5 more threads
INFO - 02:14:34: worker thread finished; awaiting finish of 4 more threads
INFO - 02:14:34: worker thread fi

Time to train the model: 0.89 mins


In [96]:
w2v_model.save("word2vec6.model")

INFO - 16:25:00: saving Word2Vec object under word2vec6.model, separately None
INFO - 16:25:00: not storing attribute vectors_norm
INFO - 16:25:00: not storing attribute cum_table
INFO - 16:25:01: saved word2vec6.model


In [ ]:
df_model['text'] = df_model['text'].astype('str')

In [88]:
#Exporting preprocessed dataset for further steps (with replaced bigrams)
file_export = df_model.copy()
file_export['old_text'] = file_export.text
file_export.head()
#file_export.old_text = file_export.old_text.str.join(' ')
#file_export.text = file_export.text.apply(lambda x: ' '.join(bigram[x]))

,text,created_at,old_text
0,"['and', 'the', 'bad', 'news', 'you', ""won't"", ...",2021-09-04T08:15:55+0000,"['and', 'the', 'bad', 'news', 'you', ""won't"", ..."
1,"['if', 'people', 'are', 'expressing', 'concern...",2021-09-04T08:11:13+0000,"['if', 'people', 'are', 'expressing', 'concern..."
2,"['how', 'about', 'our', 'kids', 'under', '12',...",2021-09-04T08:00:19+0000,"['how', 'about', 'our', 'kids', 'under', '12',..."
3,"['my', 'past', 'reading', 'n', 'understanding'...",2021-09-04T08:22:41+0000,"['my', 'past', 'reading', 'n', 'understanding'..."
4,"['protect', 'below', '12', 'year', 'olds', ""do...",2021-09-04T09:57:54+0000,"['protect', 'below', '12', 'year', 'olds', ""do..."


In [89]:
file_export['old_text'] = file_export['old_text'].apply(eval).apply(' '.join)
file_export

,text,created_at,old_text
0,"['and', 'the', 'bad', 'news', 'you', ""won't"", ...",2021-09-04T08:15:55+0000,and the bad news you won't know if you have it...
1,"['if', 'people', 'are', 'expressing', 'concern...",2021-09-04T08:11:13+0000,if people are expressing concerns then it is a...
2,"['how', 'about', 'our', 'kids', 'under', '12',...",2021-09-04T08:00:19+0000,how about our kids under 12 ? they got no prot...
3,"['my', 'past', 'reading', 'n', 'understanding'...",2021-09-04T08:22:41+0000,my past reading n understanding is that now ou...
4,"['protect', 'below', '12', 'year', 'olds', ""do...",2021-09-04T09:57:54+0000,protect below 12 year olds don't sign them up ...
...,...,...,...
56285,"['enforce', '40', 'of', 'the', 'drinks', 'offe...",2019-01-20T11:54:46+0000,enforce 40 of the drinks offer at vending mach...
56286,"['1', 'skip', 'the', 'sweet', 'sauce', 'for', ...",2019-01-20T11:34:10+0000,1 skip the sweet sauce for popiah it is tasty ...
56287,"['also', 'educate', 'the', 'true', 'source', '...",2019-01-24T23:36:28+0000,also educate the true source of health hazards...
56288,"['fine', 'how', 'much', '?', 'or', 'jus', 'war...",2019-01-01T13:58:27+0000,fine how much ? or jus warning shiok shiok only


In [90]:
file_export['text'] = file_export['text'].apply(eval).apply(lambda x: ' '.join(bigram[x]))

In [91]:
file_export

,text,created_at,old_text
0,and the bad_news you won't know if you have it...,2021-09-04T08:15:55+0000,and the bad news you won't know if you have it...
1,if people are expressing concerns then it is a...,2021-09-04T08:11:13+0000,if people are expressing concerns then it is a...
2,how about our kids under 12 ? they got no prot...,2021-09-04T08:00:19+0000,how about our kids under 12 ? they got no prot...
3,my past reading n understanding is that now ou...,2021-09-04T08:22:41+0000,my past reading n understanding is that now ou...
4,protect below 12 year olds don't sign them up ...,2021-09-04T09:57:54+0000,protect below 12 year olds don't sign them up ...
...,...,...,...
56285,enforce 40 of the drinks offer at vending mach...,2019-01-20T11:54:46+0000,enforce 40 of the drinks offer at vending mach...
56286,1 skip the sweet sauce for popiah it is tasty ...,2019-01-20T11:34:10+0000,1 skip the sweet sauce for popiah it is tasty ...
56287,also educate the true source of health hazards...,2019-01-24T23:36:28+0000,also educate the true source of health hazards...
56288,fine how much ? or jus warning shiok_shiok only,2019-01-01T13:58:27+0000,fine how much ? or jus warning shiok shiok only


In [57]:
file_export.to_csv('cleaned_dataset5.csv', index=False)